In [1]:
import keras
import os
from PIL import Image
import numpy as np
import tensorflow as tf
import json


2025-03-09 17:34:47.037077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741538087.133022  134662 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741538087.168631  134662 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 17:34:47.304862: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = keras.saving.load_model(os.path.join("", "beans_disease_classification_transfer_learning.keras"))

I0000 00:00:1741538095.874344  134662 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2874 MB memory:  -> device: 0, name: Quadro P600, pci bus id: 0000:01:00.0, compute capability: 6.1


In [3]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-s (Functional)   │ (None, 7, 7, 1280)     │    20,331,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │         3,843 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,127,034 (191.22 MB)

 Trainable params: 14,895,915 (56.82 MB)

 Non-trainable params: 5,439,288 (20.75 MB)

 Optimizer params: 29,791,831 (113.65 MB)

In [4]:
images = [Image.open(os.path.join("samples", image_path)) for image_path in os.listdir("samples")]
images

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x500>]

In [5]:
np.array(images).shape

(3, 500, 500, 3)

In [6]:
predictions = model.predict(np.array(images))
predictions

I0000 00:00:1741538106.160462  134765 cuda_dnn.cc:529] Loaded cuDNN version 90700


1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


array([[6.43029271e-05, 9.70276364e-04, 9.98965383e-01],
       [9.99979258e-01, 9.04849730e-06, 1.16682795e-05],
       [1.95633476e-07, 9.99976635e-01, 2.31454069e-05]], dtype=float32)

In [7]:
predictions = tf.where(predictions < 0.5, 0, 1)


In [8]:
predictions

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]], dtype=int32)>

In [9]:
print('Predictions:\n', predictions.numpy())

Predictions:
 [[0 0 1]
 [1 0 0]
 [0 1 0]]


In [10]:
with open(os.path.join("", "config.json")) as config:
        config = json.load(config)
id2label = config["id2label"]

In [11]:
preds = model.predict(np.array(images))
preds = preds.tolist()
labels = [
    {"label": str(id2label["0"]), "score": preds[0]},
    {"label": str(id2label["1"]), "score": preds[1]},
    {"label": str(id2label["2"]), "score": preds[2]},
]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step


In [12]:
labels

[{'label': 'angular_leaf_spot',
  'score': [6.430292705772445e-05, 0.0009702763636596501, 0.9989653825759888]},
 {'label': 'bean_rust',
  'score': [0.9999792575836182,
   9.048497304320335e-06,
   1.1668279512377921e-05]},
 {'label': 'healthy',
  'score': [1.956334756414435e-07, 0.999976634979248, 2.314540688530542e-05]}]